# Regression Week 5: Feature Selection and LASSO (Interpretation)

In this notebook, you will use LASSO to select features, building on a pre-implemented solver for LASSO (using GraphLab Create, though you can use other solvers). You will:
* Run LASSO with different L1 penalties.
* Choose best L1 penalty using a validation set.
* Choose best L1 penalty using a validation set, with additional constraint on the size of subset.

In the second notebook, you will implement your own LASSO solver, using coordinate descent. 

# Fire up graphlab create

In [1]:
import graphlab

# Load in house sales data

Dataset is from house sales in King County, the region where the city of Seattle, WA is located.

In [2]:
sales = graphlab.SFrame('kc_house_data.gl/')

2016-04-10 20:10:33,991 [INFO] graphlab.cython.cy_server, 176: GraphLab Create v1.8.5 started. Logging: /tmp/graphlab_server_1460344232.log


This non-commercial license of GraphLab Create is assigned to ankur@malloc64.com and will expire on February 01, 2017. For commercial licensing options, visit https://dato.com/buy/.


# Create new features

As in Week 2, we consider features that are some transformations of inputs.

In [3]:
from math import log, sqrt
sales['sqft_living_sqrt'] = sales['sqft_living'].apply(sqrt)
sales['sqft_lot_sqrt']    = sales['sqft_lot'].apply(sqrt)
sales['bedrooms_square']  = sales['bedrooms'] * sales['bedrooms']

# In the dataset, 'floors' was defined with type string, 
# so we'll convert them to float, before creating a new feature.
sales['floors']           = sales['floors'].astype(float) 
sales['floors_square']    = sales['floors'] * sales['floors']

* Squaring bedrooms will increase the separation between not many bedrooms (e.g. 1) and lots of bedrooms (e.g. 4) since 1^2 = 1 but 4^2 = 16. Consequently this variable will mostly affect houses with many bedrooms.
* On the other hand, taking square root of sqft_living will decrease the separation between big house and small house. The owner may not be exactly twice as happy for getting a house that is twice as big.

# Learn regression weights with L1 penalty

Let us fit a model with all the features available, plus the features we just created above.

In [4]:
all_features = ['bedrooms', 'bedrooms_square',
                'bathrooms',
                'sqft_living', 'sqft_living_sqrt',
                'sqft_lot', 'sqft_lot_sqrt',
                'floors', 'floors_square',
                'waterfront', 'view', 'condition', 'grade',
                'sqft_above',
                'sqft_basement',
                'yr_built', 'yr_renovated']

Applying L1 penalty requires adding an extra parameter (`l1_penalty`) to the linear regression call in GraphLab Create. (Other tools may have separate implementations of LASSO.)  Note that it's important to set `l2_penalty=0` to ensure we don't introduce an additional L2 penalty.

In [6]:
model_all = graphlab.linear_regression.create(sales, target='price', features=all_features, validation_set=None, 
                                              l2_penalty=0., l1_penalty=1e10, verbose=False)

Find what features had non-zero weight.

In [15]:
non_zero_coefficients = model_all.get('coefficients')[model_all.get('coefficients')['value'] != 0]
len(non_zero_coefficients)
non_zero_coefficients

name,index,value,stderr
(intercept),None,274873.05595,None
bathrooms,None,8468.53108691,None
sqft_living,None,24.4207209824,None
sqft_living_sqrt,None,350.060553386,None
grade,None,842.068034898,None
sqft_above,None,20.0247224171,None


Note that a majority of the weights have been set to zero. So by setting an L1 penalty that's large enough, we are performing a subset selection. 

***QUIZ QUESTION***:
According to this list of weights, which of the features have been chosen? 

# Selecting an L1 penalty

To find a good L1 penalty, we will explore multiple values using a validation set. Let us do three way split into train, validation, and test sets:
* Split our sales data into 2 sets: training and test
* Further split our training data into two sets: train, validation

Be *very* careful that you use seed = 1 to ensure you get the same answer!

In [16]:
# initial train/test split
(training_and_validation, testing) = sales.random_split(.9,seed=1)
# split training into train and validate
(training, validation)             = training_and_validation.random_split(0.5, seed=1)

Next, we write a loop that does the following:
* For `l1_penalty` in [10^1, 10^1.5, 10^2, 10^2.5, ..., 10^7] (to get this in Python, type `np.logspace(1, 7, num=13)`.)
    * Fit a regression model with a given `l1_penalty` on TRAIN data. Specify `l1_penalty=l1_penalty` and `l2_penalty=0.` in the parameter list.
    * Compute the RSS on VALIDATION data (here you will want to use `.predict()`) for that `l1_penalty`
* Report which `l1_penalty` produced the lowest RSS on validation data.

When you call `linear_regression.create()` make sure you set `validation_set = None`.

Note: you can turn off the print out of `linear_regression.create()` with `verbose = False`

In [33]:
import numpy as np

lowest_rss = float('inf')
lowest_l1_penalty = float('inf')
best_model = None

for l1_penalty in np.logspace(1, 7, num=13):
    # fit a model with given l1_penalty
    cur_model = graphlab.linear_regression.create(training, target='price', features=all_features, verbose=False,
                                                  validation_set=None, l2_penalty=0., l1_penalty=l1_penalty)
    # compute the rss on `validation` data
    y = validation['price']
    ycap = cur_model.predict(validation)
    residuals = y - ycap
    rss = (residuals**2).sum()
    if rss < lowest_rss:
        lowest_l1_penalty = l1_penalty
        lowest_rss = rss
        best_model = cur_model
print "lowest rss", lowest_rss, "for l1_penalty", lowest_l1_penalty

lowest rss 6.25766285142e+14 for l1_penalty 10.0


*** QUIZ QUESTIONS ***
1. What was the best value for the `l1_penalty`?
2. What is the RSS on TEST data of the model with the best `l1_penalty`?

1. Best value for `l1_penalty` = **10.0**
2. RSS on `test` data of the model with the best `l1_penalty` = **6.25766285142e+14**

***QUIZ QUESTION***
Also, using this value of L1 penalty, how many nonzero weights do you have?

In [37]:
non_zero_best_model_coeff = best_model.get('coefficients')[best_model.get('coefficients')['value'] != 0]
print len(non_zero_best_model_coeff)

18


Number of non-zero weights = **18**

# Limit the number of nonzero weights

What if we absolutely wanted to limit ourselves to, say, 7 features? This may be important if we want to derive "a rule of thumb" --- an interpretable model that has only a few features in them.

In this section, you are going to implement a simple, two phase procedure to achive this goal:
1. Explore a large range of `l1_penalty` values to find a narrow region of `l1_penalty` values where models are likely to have the desired number of non-zero weights.
2. Further explore the narrow region you found to find a good value for `l1_penalty` that achieves the desired sparsity.  Here, we will again use a validation set to choose the best value for `l1_penalty`.

In [71]:
max_nonzeros = 7

## Exploring the larger range of values to find a narrow range with the desired sparsity

Let's define a wide range of possible `l1_penalty_values`:

In [39]:
l1_penalty_values = np.logspace(8, 10, num=20)
l1_penalty_values

array([  1.00000000e+08,   1.27427499e+08,   1.62377674e+08,
         2.06913808e+08,   2.63665090e+08,   3.35981829e+08,
         4.28133240e+08,   5.45559478e+08,   6.95192796e+08,
         8.85866790e+08,   1.12883789e+09,   1.43844989e+09,
         1.83298071e+09,   2.33572147e+09,   2.97635144e+09,
         3.79269019e+09,   4.83293024e+09,   6.15848211e+09,
         7.84759970e+09,   1.00000000e+10])

Now, implement a loop that search through this space of possible `l1_penalty` values:

* For `l1_penalty` in `np.logspace(8, 10, num=20)`:
    * Fit a regression model with a given `l1_penalty` on TRAIN data. Specify `l1_penalty=l1_penalty` and `l2_penalty=0.` in the parameter list. When you call `linear_regression.create()` make sure you set `validation_set = None`
    * Extract the weights of the model and count the number of nonzeros. Save the number of nonzeros to a list.
        * *Hint: `model['coefficients']['value']` gives you an SArray with the parameters you learned.  If you call the method `.nnz()` on it, you will find the number of non-zero parameters!* 

In [44]:
nnz_counts = {}
for l1_penalty in l1_penalty_values:
    # fit a model with given l1_penalty
    model = graphlab.linear_regression.create(training, target='price', features=all_features, verbose=False,
                                              validation_set=None, l2_penalty=0., l1_penalty=l1_penalty)
    nnz_counts[l1_penalty] = model['coefficients']['value'].nnz()

Out of this large range, we want to find the two ends of our desired narrow range of `l1_penalty`.  At one end, we will have `l1_penalty` values that have too few non-zeros, and at the other end, we will have an `l1_penalty` that has too many non-zeros.  

More formally, find:
* The largest `l1_penalty` that has more non-zeros than `max_nonzero` (if we pick a penalty smaller than this value, we will definitely have too many non-zero weights)
    * Store this value in the variable `l1_penalty_min` (we will use it later)
* The smallest `l1_penalty` that has fewer non-zeros than `max_nonzero` (if we pick a penalty larger than this value, we will definitely have too few non-zero weights)
    * Store this value in the variable `l1_penalty_max` (we will use it later)


*Hint: there are many ways to do this, e.g.:*
* Programmatically within the loop above
* Creating a list with the number of non-zeros for each value of `l1_penalty` and inspecting it to find the appropriate boundaries.

In [82]:
l1_penalty_min = max([i[0] for i in nnz_counts.items() if i[1] > max_nonzeros])
l1_penalty_max = min([i[0] for i in nnz_counts.items() if i[1] < max_nonzeros])

print l1_penalty_min
print l1_penalty_max

2976351441.63
3792690190.73


***QUIZ QUESTIONS***

What values did you find for `l1_penalty_min` and`l1_penalty_max`? 

l1_penalty_min = **2976351441.63**

l1_penalty_max = **3792690190.73**

## Exploring the narrow range of values to find the solution with the right number of non-zeros that has lowest RSS on the validation set 

We will now explore the narrow region of `l1_penalty` values we found:

In [64]:
l1_penalty_values = np.linspace(l1_penalty_min,l1_penalty_max,20)

* For `l1_penalty` in `np.linspace(l1_penalty_min,l1_penalty_max,20)`:
    * Fit a regression model with a given `l1_penalty` on TRAIN data. Specify `l1_penalty=l1_penalty` and `l2_penalty=0.` in the parameter list. When you call `linear_regression.create()` make sure you set `validation_set = None`
    * Measure the RSS of the learned model on the VALIDATION set

Find the model that the lowest RSS on the VALIDATION set and has sparsity *equal* to `max_nonzero`.

In [70]:
import numpy as np

lowest_rss = float('inf')
lowest_l1_penalty = float('inf')
best_model = None

for l1_penalty in l1_penalty_values:
    # fit a model with given l1_penalty
    model = graphlab.linear_regression.create(training, target='price', features=all_features, verbose=False,
                                                  validation_set=None, l2_penalty=0., l1_penalty=l1_penalty)
    # compute the rss on `validation` data
    y = validation['price']
    ycap = model.predict(validation)
    residuals = y - ycap
    rss = (residuals**2).sum()
    sparsity = model['coefficients']['value'].nnz()
    if sparsity == max_nonzeros:
        print "(rss,l1_penalty,sparsity)=", (rss, l1_penalty, sparsity)
        if rss < lowest_rss:
            lowest_l1_penalty = l1_penalty
            lowest_rss = rss
            best_model = model
            pass
        pass
    pass
print "lowest: rss", lowest_rss, "l1_penalty", lowest_l1_penalty, "sparsity", best_model['coefficients']['value'].nnz()

(rss,l1_penalty,sparsity)= (966925692362085.2, 2976351441.6313133, 10)
(rss,l1_penalty,sparsity)= (974019450084554.6, 3019316638.9524155, 10)
(rss,l1_penalty,sparsity)= (981188367942453.8, 3062281836.2735176, 10)
(rss,l1_penalty,sparsity)= (989328342459473.5, 3105247033.5946198, 10)
(rss,l1_penalty,sparsity)= (998783211265890.1, 3148212230.9157219, 10)
(rss,l1_penalty,sparsity)= (1008477167020094.4, 3191177428.236824, 10)
(rss,l1_penalty,sparsity)= (1018298780553819.4, 3234142625.5579262, 10)
(rss,l1_penalty,sparsity)= (1028247992205977.0, 3277107822.8790283, 10)
(rss,l1_penalty,sparsity)= (1034616909232827.4, 3320073020.2001305, 8)
(rss,l1_penalty,sparsity)= (1038554735941040.2, 3363038217.5212326, 8)
(rss,l1_penalty,sparsity)= (1043237237871701.2, 3406003414.8423347, 8)
(rss,l1_penalty,sparsity)= (1046937488751711.5, 3448968612.1634369, 7)
(rss,l1_penalty,sparsity)= (1051147625612862.5, 3491933809.484539, 7)
(rss,l1_penalty,sparsity)= (1055992735342999.4, 3534899006.8056412, 7)
(rss,

***QUIZ QUESTIONS***
1. What value of `l1_penalty` in our narrow range has the lowest RSS on the VALIDATION set and has sparsity *equal* to `max_nonzeros`?
2. What features in this model have non-zero coefficients?

In [84]:
print "Lowest RSS =", lowest_rss
print "Lowest RSS l1_penalty =", lowest_l1_penalty
print "Lowest RSS sparsity model coefficients"
sparse_features = best_model['coefficients'][best_model['coefficients']['value'] != 0]
print len(sparse_features)
sparse_features

Lowest RSS = 1.04693748875e+15
Lowest RSS l1_penalty = 3448968612.16
Lowest RSS sparsity model coefficients
7


name,index,value,stderr
(intercept),None,222253.192544,None
bedrooms,None,661.722717782,None
bathrooms,None,15873.9572593,None
sqft_living,None,32.4102214513,None
sqft_living_sqrt,None,690.114773313,None
grade,None,2899.42026975,None
sqft_above,None,30.0115753022,None
